# 1b.EDA-TargetGroup
dept_id = `FOODS_3`를 조금 더 EDA 해 본다.

In [ ]:
# Import required library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )

%matplotlib inline

In [ ]:
# Create Dataframes
df_sales = pd.read_csv("./data/sales_train_validation.csv")
df_prices = pd.read_csv("./data/sell_prices.csv")
df_calendar = pd.read_csv("./data/calendar.csv")

# Target Group : dept_id = FOODS_3

### dept_id = FOODS_3 만 추출

In [ ]:
df_sales_foods_3 = df_sales[df_sales["dept_id"] == "FOODS_3"]
df_sales_foods_3.head()

### Dataframe을 Merge한다.
- Jupyter notebook의 Memory가 충분하지 않다면 Out of Memory가 발생한다

In [ ]:
df_sales_foods_3.head()

In [ ]:
# pandas.melt를 사용하여 df_sales을 merge한다.
# df_sales의 d_1 ~ d_1914 컬럼을 "sales" 컬럼의 Row들로 변환한다.
df_sales_melt = pd.melt(df_sales_foods_3, \
                        id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                        var_name='day', value_name='sales')

In [ ]:
# df_sales_melt와 df_calendar Merge
df_merged = pd.merge(df_sales_melt, df_calendar, \
                     left_on="day", \
                     right_on="d", \
                     how="left")

In [ ]:
# df_merged와 df_prices Merge
df_merged = pd.merge(df_merged, df_prices, \
                     left_on=["store_id", "item_id", "wm_yr_wk"], \
                     right_on=["store_id", "item_id", "wm_yr_wk"], \
                     how="left")

In [ ]:
# "date" column을 index로 지정
# 3분 정도 소요 (ml.m5.2xlarge 기준)
df_merged.date = df_merged.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df_merged.set_index("date", inplace=True)

In [ ]:
df_merged.head()

# EDA for dept_id = FOODS_3

### # of Unique items in FOODS_3

In [ ]:
# Unique items : 8,230
len(df_merged["id"].unique())

### Daily 판매량 평균의 분포

In [ ]:
sales_avg = pd.pivot_table(df_merged, index="id", values=["sales"]).plot(kind="hist", figsize=(12,8), grid=True)

- Observations
 - 2011-01-29 ~ 2016-06-19 Daily 판매량 평균이 10개 미만인 아이템이 거의 대부분이다.

### 전체 기간 (2011-01-29 ~ 2016-06-19) 판매량 전체 평균의 분포

In [ ]:
sales_total = pd.pivot_table(df_merged, index="id", values=["sales"], aggfunc=[np.sum]).plot(kind="hist", figsize=(12,8), grid=True)

- Observations
 - Daily 판매량 평균의 분포와 비슷하다.

### Correlation 분석

In [ ]:
import seaborn as sn

corrMatt = df_merged[["sales", "sell_price", "snap_CA", "snap_TX", "snap_WI"]].corr()
fig,ax= plt.subplots()
fig.set_size_inches(10,10)
sn.heatmap(corrMatt, square=True, annot=True, cmap="RdYlBu_r", vmin = -1, vmax = 1)

- Observations
 - "sales"와 강한 상관관계(일반적으로 0.7)가 있는 Time Series는 없다

### Seasonality

In [ ]:
fig = plt.figure(figsize=(20, 8))

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax1.title.set_text('Yearly Trend')
ax1.plot(df_merged["sales"].resample("1Y").sum())
ax1.grid()

ax2.title.set_text('Quarterly Trend')
ax2.plot(df_merged["sales"].resample("1Q").sum())
ax2.grid()

ax3.title.set_text('Monthly Trend')
ax3.plot(df_merged["sales"].resample("1M").sum())
ax3.grid()

ax4.title.set_text('Weekly Trend')
ax4.plot(df_merged["sales"].resample("1W").sum())
ax4.grid()

- Observations
    - 2012년, 2016년 데이터가 불완전 하므로 Yearly Trend는 판별학기 어렵다.
    - Quarterly, Monthly Trend는 식별 된다.

### 2015년 이후 데이터만 Visualization

In [ ]:
df_merged_2015 = df_merged.loc["2015-01-01":"2015-12-31"]

In [ ]:
df_merged_2015.head()

In [ ]:
fig = plt.figure(figsize=(20, 8))

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax1.title.set_text('Yearly Trend')
ax1.plot(df_merged_2015["sales"].resample("1Y").sum())
ax1.grid()

ax2.title.set_text('Quarterly Trend')
ax2.plot(df_merged_2015["sales"].resample("1Q").sum())
ax2.grid()

ax3.title.set_text('Monthly Trend')
ax3.plot(df_merged_2015["sales"].resample("1M").sum())
ax3.grid()

ax4.title.set_text('Weekly Trend')
ax4.plot(df_merged_2015["sales"].resample("1W").sum())
ax4.grid()

- Observatations
    - Weekly Seaonality가 가장 뚜렷하다.
    - Sales는 2015년 8월에 가장 높고 12월에 가장 낮다.

### 요일 별 sales

In [ ]:
df_saleas_by_weekday = pd.DataFrame(df_merged.groupby("weekday").sum().sales.reset_index())
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_saleas_by_weekday["weekday"] = pd.Categorical(df_saleas_by_weekday["weekday"], categories=cats, ordered=True)
df_saleas_by_weekday = df_saleas_by_weekday.sort_values("weekday")


fig = plt.figure(figsize=(10, 6))

plt.title("Sales by Weekdays")
plt.bar(cats, df_saleas_by_weekday.sales)

- Observations
    - 주말 sales가 평일보다 많다.
    - 평일 중 월요일, 금요일 sales가 가장 크다

### Daily Sales 분포 확인 - Box Plot
- Count of Daily Sales (df_merged["sales"] : 15743990
- Mean of Daily Sales (df_merged["sales"] : 2.0561545072119585
- Min of Daily Sales (df_merged["sales"] : 0
- Max of Daily Sales (df_merged["sales"] : 763
- Std of Daily Sales (df_merged["sales"] : 6.320518901897881

In [ ]:
df_merged.head()

In [ ]:
df_merged.boxplot(column=["sales"], by=["month"], sym="bo", figsize=(20,10))
plt.show()

- Observations
    - 3Q(Quantile)을 벗어나는 Outliner가 많다.

### Event가 Sales에 미치는 영향

In [ ]:
# 1일 평균 sales 수량
avg_sales_all = df_merged["sales"].sum() / len(df_merged)

# Event시 평균 sales 수량
df_merged_wo_nan = df_merged.dropna(subset=["event_name_1"])
avg_sales_event = df_merged_wo_nan["sales"].sum() / len(df_merged_wo_nan)
print("Everage Daily Sales : {}, Everage Daily Sales on Event : {}".format(avg_sales_all, avg_sales_event))

- Observations
    - event 했을 때 평균 Sales 수량은 event를 하지 않았을 때 보다 오히려 낮다 -> Event가 Sales에 영향을 주지 않았다라고 볼 수 있다.

In [ ]:
# event_name_1에서 Unique value 추출
events = df_merged["event_name_1"].dropna().unique()

df_sales_by_event = pd.DataFrame(columns=["event_name", "sales"])
for event in events:
    sales_by_event = df_merged_wo_nan[df_merged_wo_nan["event_name_1"] == event]["sales"].sum()
    df_sales_by_event = df_sales_by_event.append(pd.DataFrame([[event, sales_by_event]], columns=["event_name", "sales"]))
df_sales_by_event = df_sales_by_event.sort_values(by="sales")

idx = df_sales_by_event.event_name
plt.figure(figsize=(20, 15))
plt.barh(idx, df_sales_by_event["sales"])
plt.title("Sales by each Event", fontsize=18)

- Observations
    - SuperBowl event때 가장 많은 sales가 크다.
    - NBAFinalsStart + NBAFinalsEnd는 단일 event가 아닌 event series이므로 합산할 경우 SuperBowl sales보다 크다.
    - Christmas의 sales는 "72", 거의 없다.
    - SuperBowl, LaborDay, Purim End, Easter, NBAFinalsStart+NBAFinalsEnd의 total sales가 100,000 이상이다.

In [ ]:
df_sales_by_event.sort_values(by="sales", ascending=False)

### SuperBowl, LaborDay, Purim End, Easter, NBAFinalsStart, NBAFinalsEnd 날짜

In [ ]:
events = ["SuperBowl", "LaborDay", "Purim End", "Easter", "NBAFinalsStart", "NBAFinalsEnd"]
df_merged[df_merged["event_name_1"].isin(events)][["event_name_1", "wm_yr_wk"]]\
    .drop_duplicates()\
    .sort_values(by=["event_name_1", "wm_yr_wk"])

- Observations
    - SuperBowl은 매년 2월 초에 한다.

### Plots

In [ ]:
# Daily sales가 가장 많은 item Top10 list, 가장 적은 item Worst10 list 선택
df_merged_best10  = df_merged.groupby("id").sum().sort_values(by="sales", ascending=False).reset_index().head(10)
df_merged_worst10 = df_merged.groupby("id").sum().sort_values(by="sales", ascending=False).reset_index().tail(10)

In [ ]:
# 전체 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc[:]["sales"].plot(figsize=(15,5), grid=True)

- Observations
    - 2013년까지는 연말에 Spiky했으나 2014년 이후에는 연말에 Spiky패턴이 보이지 않는다.
    - Daily판매량("sales")는 Linear하게 증가하지 않는다.

In [ ]:
# 2011년 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2011-01-01":"2011-12-31"]["sales"].plot(figsize=(15,5), grid=True)

In [ ]:
# 2012년 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2012-01-01":"2012-12-31"]["sales"].plot(figsize=(15,5), grid=True)

In [ ]:
# 2013년 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2013-01-01":"2013-12-31"]["sales"].plot(figsize=(15,5), grid=True)

In [ ]:
# 2014년 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2014-01-01":"2014-12-31"]["sales"].plot(figsize=(15,5), grid=True)

In [ ]:
# 2015년 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2015-01-01":"2015-12-31"]["sales"].plot(figsize=(15,5), grid=True)

In [ ]:
# 2016년 이후 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2016-01-01":]["sales"].plot(figsize=(15,5), grid=True)

### Best10의 Seasonality

In [ ]:
fig = plt.figure(figsize=(30, 8))

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

for item_id in df_merged_best10["id"]:
    ax1.title.set_text('Yearly Trend')
    ax1.plot(df_merged[df_merged["id"] == item_id]["sales"].resample("1Y").sum())
    ax1.grid()

    ax2.title.set_text('Quarterly Trend')
    ax2.plot(df_merged[df_merged["id"] == item_id]["sales"].resample("1Q").sum())
    ax2.grid()

    ax3.title.set_text('Monthly Trend')
    ax3.plot(df_merged[df_merged["id"] == item_id]["sales"].resample("1M").sum())
    ax3.grid()

    ax4.title.set_text('Weekly Trend')
    ax4.plot(df_merged[df_merged["id"] == item_id]["sales"].resample("1W").sum())
    ax4.grid()

- Observations
 - Best10의 sales volume은 감소 trend.

In [ ]:
# 2013년 Plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2015-11-01":"2016-01-31"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

- Observations
    - 2013년에 Sales볼륨이 굉장히 컸다.
    - 2013-08-15 ~ 2013-09-15 사이 Sales가 증가 했다.

In [ ]:
# SuperBowl (2012-02-05)의 sales plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2012-01-01":"2012-02-28"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

In [ ]:
# NBA Final(2013-06-06 ~ 2013-06-20)의 sales plot
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2013-05-01":"2013-07-31"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

In [ ]:
# Black Friday(매년 11월 말) ~ 다음 해 1월까지 sales plot, 2012년
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2012-11-01":"2013-01-31"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

In [ ]:
# Black Friday(매년 11월 말) ~ 다음 해 1월까지 sales plot, 2013년
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2013-11-01":"2014-01-31"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

In [ ]:
# Black Friday(매년 11월 말) ~ 다음 해 1월까지 sales plot, 2014년
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2014-11-01":"2015-01-31"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

In [ ]:
# Black Friday(매년 11월 말) ~ 다음 해 1월까지 sales plot, 2015년
for item_id in df_merged_best10["id"]:
    df_merged[df_merged["id"] == item_id].loc["2015-11-01":"2016-01-31"]["sales"].plot(figsize=(20,5), grid=True, label=item_id)

- Observations
    - Black Friday기간에 Sales는 상승하는 것 처럼 보인다.
    - Christmas기간에 Sales는 0에 가깝다.
    - 2013년, 2014년 1월 1일은 Sales가 0에 가깝지만 2015, 2016년 1월 1일은 다른 Trend

### Worst10 분석

In [ ]:
df_merged_worst10

In [ ]:
# Plot
for item_id in df_merged_worst10["id"]:
    df_merged[df_merged["id"] == item_id]["sales"].plot(figsize=(15,5), grid=True)

- Observations
    - 거의 팔리지 않는다

In [ ]:
%store df_merged
%store df_sales_foods_3